In [1]:
!pip install pycaret --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
pydocstyle 6.3.0 requires importlib-metadata<5.0.0,>=2.0.0; python_version < "3.8", but you have importlib-metadata 6.4.1 which is incompatible.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
ibis-framework 2.1.1 requires importlib-metadata<5,>=4; python_version < "3.8", but you have importlib-metadata 6.4.1 which is incompatible.
flake8 5.0.4 requires importlib-metadata<4.3,>=1.1.0; python_version < "3.8", but you have importlib-metadata 6.4.1 which is incompatible.
cmudict 1.0.13 requires importlib-metadata<6.0.0,>=5.1.0, but you have importlib-metadata 6.4.1 which is incompatible.


In [2]:
import pandas as pd
import os

train_path = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train"
tdcsfog_path = train_path + "/tdcsfog"
defog_path = train_path + "/defog"
notype_path = train_path + "/notype"
tdcsfog_data = pd.concat([pd.read_csv(os.path.join(tdcsfog_path, i)).assign(Class='tdcsfog') for i in os.listdir(tdcsfog_path)])
defog_data = pd.concat([pd.read_csv(os.path.join(defog_path, i)).assign(Class='defog') for i in os.listdir(defog_path)])
notype_data = pd.concat([pd.read_csv(os.path.join(notype_path, i)).assign(Class='notype') for i in os.listdir(notype_path)])
train_data = pd.concat([tdcsfog_data,defog_data])
train_data['status'] = 'Others'
train_data.loc[(train_data['StartHesitation'] == 1), 'status'] = 'StartHesitation'
train_data.loc[(train_data['Turn'] == 1), 'status'] = 'Turn'
train_data.loc[(train_data['Walking'] == 1), 'status'] = 'Walking'
#train_data['Valid'] = train_data['Valid'].astype(bool)
#train_data = train_data[train_data['Valid']]
train_data = train_data.drop(['StartHesitation', 'Turn', 'Walking','Valid','Task'], axis=1)
train_data = train_data.reset_index(drop=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20588374 entries, 0 to 20588373
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Time    int64  
 1   AccV    float64
 2   AccML   float64
 3   AccAP   float64
 4   Class   object 
 5   status  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 942.5+ MB


In [3]:
train_data.status.describe()

count     20588374
unique           4
top         Others
freq      17735798
Name: status, dtype: object

In [4]:
test_path = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test"
tdcsfog_path = test_path + "/tdcsfog"
defog_path = test_path + "/defog"
tdcsfog_data = pd.concat([pd.read_csv(os.path.join(tdcsfog_path, i)).assign(Id=i.replace('.csv', '')).assign(Class='tdcsfog') for i in os.listdir(tdcsfog_path)])
defog_data = pd.concat([pd.read_csv(os.path.join(defog_path, i)).assign(Id=i.replace('.csv', '')).assign(Class='defog') for i in os.listdir(defog_path)])
test_data = pd.concat([tdcsfog_data,defog_data])
test_data = test_data.reset_index(drop=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286370 entries, 0 to 286369
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    286370 non-null  int64  
 1   AccV    286370 non-null  float64
 2   AccML   286370 non-null  float64
 3   AccAP   286370 non-null  float64
 4   Id      286370 non-null  object 
 5   Class   286370 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 13.1+ MB


In [5]:
from pycaret.classification import *
clf = setup(data = train_data, target='status',use_gpu = True)

,Description,Value
0,Session id,3838
1,Target,status
2,Target type,Multiclass
3,Target mapping,"Others: 0, StartHesitation: 1, Turn: 2, Walking: 3"
4,Original data shape,"(20588374, 6)"
5,Transformed data shape,"(20588374, 6)"
6,Transformed train set shape,"(14411861, 6)"
7,Transformed test set shape,"(6176513, 6)"
8,Ordinal features,1
9,Numeric features,4


In [6]:
catboost = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9038,0.9174,0.9038,0.8875,0.8859,0.4991,0.5330
1,0.9033,0.9165,0.9033,0.8870,0.8853,0.4947,0.5300
2,0.9036,0.9166,0.9036,0.8871,0.8855,0.4963,0.5314
3,0.9039,0.9171,0.9039,0.8877,0.8861,0.4998,0.5339
4,0.9037,0.9165,0.9037,0.8874,0.8859,0.4987,0.5328
5,0.9036,0.9162,0.9036,0.8872,0.8856,0.4967,0.5316
6,0.9038,0.9167,0.9038,0.8875,0.8859,0.4986,0.5329
7,0.9035,0.9168,0.9035,0.8871,0.8855,0.4965,0.5313
8,0.9036,0.9167,0.9036,0.8872,0.8856,0.4972,0.5319


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
xgboost = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9049,0.9244,0.9049,0.8904,0.8871,0.5035,0.5389
1,0.9046,0.9239,0.9046,0.8897,0.8868,0.5021,0.5372
2,0.9047,0.9238,0.9047,0.8901,0.8869,0.5023,0.5379
3,0.9050,0.9243,0.9050,0.8903,0.8874,0.5051,0.5399
4,0.9049,0.9242,0.9049,0.8903,0.8871,0.5036,0.5389
5,0.9048,0.9236,0.9048,0.8901,0.8868,0.5017,0.5378
6,0.9049,0.9251,0.9049,0.8901,0.8872,0.5039,0.5389
7,0.9048,0.9251,0.9048,0.8900,0.8870,0.5032,0.5383
8,0.9047,0.9246,0.9047,0.8899,0.8870,0.5034,0.5381


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
#catboost = tune_model(catboost)

In [9]:
#xgboost = tune_model(xgboost)

In [10]:
blend_xgb_cat = blend_models([catboost,xgboost])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9053,0.9243,0.9053,0.8911,0.8873,0.5044,0.5408
1,0.9049,0.9238,0.9049,0.8904,0.8867,0.5011,0.5381
2,0.9051,0.9237,0.9051,0.8906,0.8870,0.5026,0.5394
3,0.9054,0.9243,0.9054,0.8910,0.8875,0.5049,0.5414
4,0.9052,0.9239,0.9052,0.8908,0.8871,0.5035,0.5400
5,0.9051,0.9235,0.9051,0.8906,0.8869,0.5022,0.5393
6,0.9052,0.9246,0.9052,0.8907,0.8871,0.5034,0.5399
7,0.9051,0.9244,0.9051,0.8904,0.8870,0.5026,0.5392
8,0.9050,0.9243,0.9050,0.8905,0.8869,0.5023,0.5389


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
save_model(xgboost, 'xgboost')
save_model(catboost, 'catboost')
save_model(blend_xgb_cat, 'blend_xgb_cat')

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Time', 'AccV', 'AccML', 'AccAP'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               missing_values=nan,
                                                               strateg...
                                                              learning_rate=0.300000012,
                                                              max_bin=256,
                             

In [12]:
#blend_xgb_cat = finalize_model(blend_xgb_cat)

In [13]:
#test_data = predict_model(xgboost,data = test_data)

In [14]:
'''result = pd.DataFrame(columns=['Id', 'StartHesitation', 'Turn', 'Walking'])

for id in test_data['Id'].unique():
    tmp = test_data[test_data['Id'] == id]
    id_str = str(tmp['Id'].iloc[0])
    for i in range(len(tmp)):
        id_time = id_str + '_' + str(tmp['Time'].iloc[i])
        row = {'Id': id_time, 'StartHesitation': 0, 'Turn': 0, 'Walking': 0}
        label = tmp['prediction_label'].iloc[i]
        if label in ['StartHesitation', 'Turn', 'Walking']:
            row[label] = 1
        result = result.append(row, ignore_index=True)

# 保存为 csv 文件
result.to_csv('result.csv', index=False)'''

"result = pd.DataFrame(columns=['Id', 'StartHesitation', 'Turn', 'Walking'])\n\nfor id in test_data['Id'].unique():\n    tmp = test_data[test_data['Id'] == id]\n    id_str = str(tmp['Id'].iloc[0])\n    for i in range(len(tmp)):\n        id_time = id_str + '_' + str(tmp['Time'].iloc[i])\n        row = {'Id': id_time, 'StartHesitation': 0, 'Turn': 0, 'Walking': 0}\n        label = tmp['prediction_label'].iloc[i]\n        if label in ['StartHesitation', 'Turn', 'Walking']:\n            row[label] = 1\n        result = result.append(row, ignore_index=True)\n\n# 保存为 csv 文件\nresult.to_csv('result.csv', index=False)"